# Main-and-Transfer Substation Test

## Create a New Substation

### Import CIMantic Graphs Library and CIM profile

In [1]:
# Import transmission and distribution modeling classes
from cimgraph.models import FeederModel, NodeBreakerModel
from cimgraph.databases import ConnectionParameters, RDFlibConnection, BlazegraphConnection
import cimgraph.utils as utils

import importlib

cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

params = ConnectionParameters(filename=None, cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)

### Create New Main-and-Transfer Substation

In [2]:
from cimbuilder.substation_builder import BreakerAndHalfSubstation

In [3]:
SubBuilder = BreakerAndHalfSubstation(connection=connection, name="breaker_and_half_sub", base_voltage = 115000)
substation = SubBuilder.substation
print(substation)

Substation(mRID='8851b93c-b3ab-4c10-bc58-2ee77431d89e', aliasName=None, description=None, name='breaker_and_half_sub', Names=[], AssetDatasheet=None, Assets=[], ConfigurationEvent=[], Controls=[], Location=None, Measurements=[], OperatingShare=[], PSRType=None, ReportingGroup=[], ConnectivityNodes=[], TopologicalNode=[], AdditionalGroupedEquipment=[], Equipments=[], Bays=[], NamingFeeder=None, NormalEnergizedFeeder=[], NormalEnergizingFeeder=[], Region=None, VoltageLevels=[])


### Add IEEE 13 Feeder and IEEE 13 Assets Feeder to substation

In [4]:
# Import 13 bus model from XML file
ieee13_feeder = cim.Feeder(mRID = '49AD8E07-3BF9-A4E2-CB8F-C3722F837B62')
params = ConnectionParameters(filename='../test_models/IEEE13.xml', cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)
ieee13_network = FeederModel(connection=connection, container=ieee13_feeder, distributed=False)


In [5]:

assets13_feeder = cim.Feeder(mRID = '5B816B93-7A5F-B64C-8460-47C17D6E4B0F')
params = ConnectionParameters(filename='../test_models/IEEE13_Assets.xml', cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)
assets13_network = FeederModel(connection=connection, container=assets13_feeder, distributed=False)


In [6]:
SubBuilder.new_feeder(feeder_number = 1, branch_number = 1, feeder=ieee13_feeder, feeder_network=ieee13_network)
SubBuilder.new_feeder(feeder_number = 2, branch_number = 2, feeder=assets13_feeder, feeder_network=assets13_network)

In [7]:
SubBuilder.network.pprint(cim.Substation)
SubBuilder.network.pprint(cim.Feeder)

{
    "8851b93c-b3ab-4c10-bc58-2ee77431d89e": {
        "mRID": "8851b93c-b3ab-4c10-bc58-2ee77431d89e",
        "name": "breaker_and_half_sub"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "NormalEnergizingSubstation": "8851b93c-b3ab-4c10-bc58-2ee77431d89e"
    },
    "5B816B93-7A5F-B64C-8460-47C17D6E4B0F": {
        "mRID": "5B816B93-7A5F-B64C-8460-47C17D6E4B0F",
        "NormalEnergizingSubstation": "8851b93c-b3ab-4c10-bc58-2ee77431d89e"
    }
}


### Save Substation to XML File

In [8]:
utils.write_xml(SubBuilder.network, '../test_output/breaker_and_half.xml')

![breaker-and-half](./breaker_and_half.png)

## Round-Trip Test: Load and Read from Database

### Delete all old entries from database and load new models

In [9]:
# Connect to Blazegraph Database
from cimgraph.databases import BlazegraphConnection
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
blazegraph = BlazegraphConnection(params)
blazegraph.execute('drop all')

/home/singha42/.cache/pypoetry/virtualenvs/cim-builder-hXthbBAa-py3.10/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=44ms, elapsed=44ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=57ms, commitTime=1708970717031, mutationCount=6532</p\n></html\n>'

In [10]:
from cimloader.databases.blazegraph import BlazegraphConnection as BlazegraphLoader
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
loader = BlazegraphLoader(params)

In [11]:
loader.upload_from_file(filename='../test_models/IEEE13.xml')
loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
loader.upload_from_file(filename='../test_output/breaker_and_half.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="49"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="37"/>

curl: Can't open '../test_output/breaker_and_half.xml'!
curl: try 'curl --help' or 'curl --manual' for more information


### Create new NodeBreakerModel of Substation + Feeders

In [12]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [13]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "8851b93c-b3ab-4c10-bc58-2ee77431d89e": {
        "mRID": "8851b93c-b3ab-4c10-bc58-2ee77431d89e",
        "name": "breaker_and_half_sub"
    }
}


In [14]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{}


In [15]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

KeyError: <class 'cimgraph.data_profile.cimhub_2023.cimhub_profile_2023.EnergyConsumer'>

### Merge substation and feeders into a single XML file 

In [ ]:
utils.get_all_data(network)

ConnectivityNode
Terminal
Disconnector
Breaker
BusbarSection
Substation
Feeder
TransformerTank
ACLineSegment
PowerTransformer
LoadBreakSwitch
EnergyConsumer
Fuse
PowerElectronicsConnection
EnergySource
LinearShuntCompensator
Recloser
Location
BaseVoltage
LoadResponseCharacteristic
EnergyConsumerPhase
OperationalLimitSet
TopologicalNode
TapChangerControl
TransformerTankEnd
PowerTransformerEnd
RegulatingControl
WireSpacingInfo
ACLineSegmentPhase
PerLengthPhaseImpedance
OverheadWireInfo
ConcentricNeutralCableInfo
TapeShieldCableInfo
PhaseImpedanceData
WirePosition
TransformerTankInfo
RatioTapChanger
TransformerEndInfo
ShortCircuitTest
NoLoadTest
TransformerMeshImpedance
TransformerCoreAdmittance
BatteryUnit
PowerElectronicsConnectionPhase
PhotovoltaicUnit
CurrentLimit
VoltageLimit
OperationalLimitType
CoordinateSystem
PositionPoint
SubGeographicalRegion
SwitchPhase
LinearShuntCompensatorPhase
TopologicalIsland
GeographicalRegion


In [ ]:
utils.write_xml(network, '../test_output/breaker_and_half_and_feeders.xml')
